# upload to GBQ and Remove file

In [1]:
# import os, sys
import datetime
import pandas as pd
import numpy as np
import datetime
import pandas_gbq
import datetime
from google.cloud import bigquery
import win32com.client
from win32com.client import Dispatch, constants
import os
import shutil



# Open a file
path = r"\\CPNVFS04\depts\HD\IPR\Parm Sim"
##path = "C:\Users\yxg0rzh\OneDrive - The Home Depot\Desktop\New folder (2)"
dirs = os.listdir(path)
##path = r"C:\Users\yxg0rzh\OneDrive - The Home Depot\Desktop\Transfers_tl\FW 24 Due 5PM 7-13-20\COMPILED"

##This would print all the files and directories
for file in dirs:
    URL = os.path.join(path, file)

In [2]:
for file in os.listdir(path):
    URL = os.path.join(path, file)
    if os.path.isdir(URL):
        # skip directories
        continue
    print(URL)

\\CPNVFS04\depts\HD\IPR\Parm Sim\Cabinet Min Test MKT 4.csv


In [3]:
hours = 4
hours_added = datetime.timedelta(hours = hours)

In [4]:
path1 = r"\\CPNVFS04\depts\HD\IPR\Parm Sim\Archive"
combine = pd.DataFrame()
for file in dirs:
    URL = os.path.join(path, file)
    if os.path.isdir(URL):
        # skip directories
        continue
    try:
        df = pd.read_csv(URL, header=0)
        df.columns = [a.replace(" ","_") for a in df.columns.values]
    except:
        df = pd.read_excel(URL, header=0)
        df.columns = [a.replace(" ","_") for a in df.columns.values]
    df = df.astype({'Trumping_Level': 'int64', 
               'Type': 'int64', 
               'Store_Group':'float64', 
               'Volume_Id':'float64',
               'Velocity_Id':'str',
               'SKU':'float64',
               'SKU_Grp':'float64',
               'Store': 'float64',
               'Parm_code':'int64',
               'Parm_Value':'float64',
               'Eff_Begin_date':'object',
               'Eff_End_date':  'object',
               'Start_Fscl_Wk_nbr':'float64',
               'End_Fscl_Wk_nbr':'float64',
               'Param_Desc' : 'object',
               'OOTL_Reason_Code' :'object'})
    df["time"] = datetime.datetime.now() + hours_added

    
    
    combine = pd.concat([combine, df],ignore_index=True)
    combine = combine.dropna(how='all')
    shutil.move(URL, os.path.join(path1, file))
    
print (URL, combine.dtypes)
print("Loaded {} rows into {}:{}.".format(len(combine),'analytics-supplychain-thd', "akaboshi_test.sam_test2"))
#pandas_gbq.to_gbq(combine, "akaboshi_test.Aug24th", if_exists = "replace" ,project_id='analytics-supplychain-thd')
pandas_gbq.to_gbq(combine, "akaboshi_test.sam_test2", if_exists = "replace" ,project_id='analytics-supplychain-thd')


\\CPNVFS04\depts\HD\IPR\Parm Sim\Cabinet Min Test MKT 4.csv Trumping_Level                int64
Type                          int64
Store_Group                 float64
Volume_Id                   float64
Velocity_Id                  object
SKU                         float64
SKU_Grp                     float64
Store                       float64
Parm_code                     int64
Parm_Value                  float64
Eff_Begin_date               object
Eff_End_date                 object
Start_Fscl_Wk_nbr           float64
End_Fscl_Wk_nbr             float64
Param_Desc                   object
OOTL_Reason_Code             object
time                 datetime64[ns]
dtype: object
Loaded 6389 rows into analytics-supplychain-thd:akaboshi_test.sam_test2.


C:\Users\yxg0rzh\AppData\Local\Continuum\anaconda3\lib\site-packages\google\auth\_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
1it [00:00,  6.05it/s]


# Send email

In [5]:
from oauth2client.client import GoogleCredentials
from googleapiclient import discovery

## reads BQ tables into Data Frames
from pandas_gbq import read_gbq 
from pandas_gbq import to_gbq
from pandas import DataFrame 
from datetime import date 
#credentials = GoogleCredentials.get_application_default()
#bq = discovery.build('bigquery', 'v2', credentials=credentials)
query = "SELECT  distinct LDAP,EMAIL, CRT_TS from SXR_PARM_SIM.IMPACT_SUMMARY where CRT_TS >= TIMESTAMP_ADD(CURRENT_TIMESTAMP(), INTERVAL -10 MINUTE)"
#query = " SELECT  distinct LDAP,EMAIL from AXM3066.parm_sim_summary_test where CRT_TS>= TIMESTAMP_SUB(CURRENT_TIMESTAMP(),INTERVAL 15 MINUTE)"
df = pd.read_gbq(query, 'analytics-supplychain-thd', dialect = 'standard')

C:\Users\yxg0rzh\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\internals\construction.py:703: FutureWarning: Converting timezone-aware DatetimeArray to timezone-naive ndarray with 'datetime64[ns]' dtype. In the future, this will return an ndarray with 'object' dtype where each element is a 'pandas.Timestamp' with the correct 'tz'.
	To accept the future behavior, pass 'dtype=object'.
	To keep the old behavior, pass 'dtype="datetime64[ns]"'.
  subarr = construct_1d_object_array_from_listlike(subarr)


In [6]:
df

,LDAP,EMAIL,CRT_TS
0,DJA57GH,DAVAN_J_ABNEY@homedepot.com,2021-02-11 15:22:09.259534+00:00


In [7]:
for index, row in df.iterrows():
    query = "SELECT * from SXR_PARM_SIM.IMPACT_SUMMARY where LDAP =" + "'"+str(row["LDAP"])+"'" + " and CRT_TS = " + "'"+str(row["CRT_TS"])+"'"
    query1 = "SELECT * from SXR_PARM_SIM.STR_SKU_SUMMARY where LDAP =" + "'"+str(row["LDAP"])+"'" + " and CRT_TS = " + "'"+str(row["CRT_TS"])+"'"
    data = pd.read_gbq(query, 'analytics-supplychain-thd', dialect = 'standard')
    path= 'C:/Users/yxg0rzh/OneDrive - The Home Depot/Desktop/New folder (2)/sam_test//'
    data.to_csv(os.path.join(path, 'sam_test ' + str(row["LDAP"]) + '.csv'),index=False)    
    
    const=win32com.client.constants
    olMailItem = 0x0
    obj = win32com.client.Dispatch("Outlook.Application")
    newMail = obj.CreateItem(olMailItem)
    newMail.Subject = "Parm Sim Results"
# newMail.Body = "I AM\nTHE BODY MESSAGE!"
    newMail.BodyFormat = 2 # olFormatHTML https://msdn.microsoft.com/en-us/library/office/aa219371(v=office.11).aspx
    newMail.HTMLBody = "<HTML><BODY>Please find results of your parameter simulation attached. Get store-SKU results with BQ query below:</BODY></HTML>"+ query1
#    newMail.To = "yunzhong_gao@homedepot.com"
    newMail.To = row["EMAIL"]
    attachment1 = os.path.join(path, 'sam_test ' + str(row["LDAP"]) + '.csv')
    newMail.Attachments.Add(Source=attachment1)
    newMail.display()
    newMail.Send()

In [26]:
for index, row in df.iterrows():
    query = "SELECT * from SXR_PARM_SIM.IMPACT_SUMMARY where LDAP =" + "'"+str(row["LDAP"])+"'" + " and CRT_TS = " + "'"+str(row["CRT_TS"])+"'"

In [11]:
query

"SELECT * from SXR_PARM_SIM.IMPACT_SUMMARY where LDAP ='AXM3066' and CRT_TS = '2020-08-12 20:24:14.343881+00:00'"

In [24]:
os.path.join(path, 'sam_test ' + str(row["LDAP"]) + '.csv')

'C:/Users/yxg0rzh/OneDrive - The Home Depot/Desktop/New folder (2)/sam_test//sam_test YXG0RZ.csv'